In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from DRflow.metrics import ABW, CAL, DSC, HM, NH, SC
from DRflow.metrics import AUClogRNX, CCA, CC, LCMC, NeRV, NLM, TandC, Stress,scagnostics

from DRflow.utils.evaluate import evaluate_files, apply_highlow, apply_supervised
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import ray
all_names = ['MNIST','coil-100','stanfordfaces','yalefaces','Caltech_instruments', 
             'Caltech_plants', 'Caltech_vehicles', 
         'fashionmnist',   'flowers',
         'paintings','oxford_buildings','paris_buildings']

# /Users/morarica/Developer/DumbleDR/public/data/{}/human_sample_mini

SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000


In [3]:
path = '../../data/{}/mini_batch/dr/'
path = '/Users/morarica/Developer/DumbleDR/public/data/{}/human_sample_mini/'
all_names
# ray.shutdown()
ray.init()
for name in all_names:
    evaluate_files(types=['highlow'], high_dir = '../../data/{}/flatfiles/'.format(name),
                      dr_dir = path.format(name), output_dir ='../../data/{}/mini_batch/metrics/'.format(name))
        
    

2020-09-02 19:25:51,539	INFO resource_spec.py:212 -- Starting Ray with 4.74 GiB memory available for workers and up to 2.37 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-02 19:25:51,885	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


MNIST_size28_c6_ISM_n5.csv
/Users/morarica/Developer/DumbleDR/public/data/MNIST/human_sample_mini/MNIST_size28_c6_ISM_n5.csv ../../data/MNIST/flatfiles/MNIST_flat28_6classes.csv ../../data/MNIST/mini_batch/metrics/
ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)
MNIST_size28_c6_GRP.csv
/Users/morarica/Developer/DumbleDR/public/data/MNIST/human_sample_mini/MNIST_size28_c6_GRP.csv ../../data/MNIST/flatfiles/MNIST_flat28_6classes.csv ../../data/MNIST/mini_batch/metrics/
ObjectID(ef0a6c221819881cffffffff010000c801000000)
MNIST_size28_c6_UMAP_n2_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/MNIST/human_sample_mini/MNIST_size28_c6_UMAP_n2_d0.1.csv ../../data/MNIST/flatfiles/MNIST_flat28_6classes.csv ../../data/MNIST/mini_batch/metrics/
ObjectID(f66d17bae2b0e765ffffffff010000c801000000)
MNIST_size28_c6_UMAP_n15_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/MNIST/human_sample_mini/MNIST_size28_c6_UMAP_n15_d0.1.csv ../../data/MNIST/flatfiles/MNIST_flat28_6classes.csv ../.

ObjectID(c6617bc1ef8049beffffffff010000c801000000)
Caltech_plants_size200_c6_UMAP_n15_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/Caltech_plants/human_sample_mini/Caltech_plants_size200_c6_UMAP_n15_d0.1.csv ../../data/Caltech_plants/flatfiles/Caltech_plants_flat50_6classes.csv ../../data/Caltech_plants/mini_batch/metrics/
ObjectID(fa0fd98ff6c4c659ffffffff010000c801000000)
Caltech_plants_size200_c6_UMAP_n7_d0.8.csv
/Users/morarica/Developer/DumbleDR/public/data/Caltech_plants/human_sample_mini/Caltech_plants_size200_c6_UMAP_n7_d0.8.csv ../../data/Caltech_plants/flatfiles/Caltech_plants_flat50_6classes.csv ../../data/Caltech_plants/mini_batch/metrics/
ObjectID(adb556579b44d151ffffffff010000c801000000)
Caltech_plants_size200_c6_UMAP_n2_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/Caltech_plants/human_sample_mini/Caltech_plants_size200_c6_UMAP_n2_d0.1.csv ../../data/Caltech_plants/flatfiles/Caltech_plants_flat50_6classes.csv ../../data/Caltech_plants/mini_batch/metri

ObjectID(bc95f5aa734639d5ffffffff010000c801000000)
fashionmnist_size50_c6_GRP.csv
/Users/morarica/Developer/DumbleDR/public/data/fashionmnist/human_sample_mini/fashionmnist_size50_c6_GRP.csv ../../data/fashionmnist/flatfiles/fashionmnist_flat50_6classes.csv ../../data/fashionmnist/mini_batch/metrics/
ObjectID(40ac1a92d080ec19ffffffff010000c801000000)
fashionmnist_size50_c6_MDS.csv
/Users/morarica/Developer/DumbleDR/public/data/fashionmnist/human_sample_mini/fashionmnist_size50_c6_MDS.csv ../../data/fashionmnist/flatfiles/fashionmnist_flat50_6classes.csv ../../data/fashionmnist/mini_batch/metrics/
ObjectID(4550d19d6cad6208ffffffff010000c801000000)
fashionmnist_size50_c6_TSNE_p100.csv
/Users/morarica/Developer/DumbleDR/public/data/fashionmnist/human_sample_mini/fashionmnist_size50_c6_TSNE_p100.csv ../../data/fashionmnist/flatfiles/fashionmnist_flat50_6classes.csv ../../data/fashionmnist/mini_batch/metrics/
ObjectID(0a3790a3760e3c5cffffffff010000c801000000)
fashionmnist_size50_c6_UMAP_n5_

ObjectID(e4e79ae86e7e7f9dffffffff010000c801000000)
flowers_size200_c6_UMAP_n5_d0.8.csv
/Users/morarica/Developer/DumbleDR/public/data/flowers/human_sample_mini/flowers_size200_c6_UMAP_n5_d0.8.csv ../../data/flowers/flatfiles/flowers_flat50_6classes.csv ../../data/flowers/mini_batch/metrics/
ObjectID(b6015f435b789377ffffffff010000c801000000)
flowers_size200_c6_GRP.csv
/Users/morarica/Developer/DumbleDR/public/data/flowers/human_sample_mini/flowers_size200_c6_GRP.csv ../../data/flowers/flatfiles/flowers_flat50_6classes.csv ../../data/flowers/mini_batch/metrics/
ObjectID(3c597180a00d601affffffff010000c801000000)
flowers_size200_c6_UMAP_n5_d0.5.csv
/Users/morarica/Developer/DumbleDR/public/data/flowers/human_sample_mini/flowers_size200_c6_UMAP_n5_d0.5.csv ../../data/flowers/flatfiles/flowers_flat50_6classes.csv ../../data/flowers/mini_batch/metrics/
ObjectID(1a6177140a5881c6ffffffff010000c801000000)
flowers_size200_c6_UMAP_n5_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/flowers/

oxford_buildings_size200_c6_UMAP_n15_d0.8.csv
/Users/morarica/Developer/DumbleDR/public/data/oxford_buildings/human_sample_mini/oxford_buildings_size200_c6_UMAP_n15_d0.8.csv ../../data/oxford_buildings/flatfiles/oxford_buildings_flat50_6classes.csv ../../data/oxford_buildings/mini_batch/metrics/
ObjectID(4ff517a2730e7822ffffffff010000c801000000)
oxford_buildings_size200_c6_SE_n7.csv
/Users/morarica/Developer/DumbleDR/public/data/oxford_buildings/human_sample_mini/oxford_buildings_size200_c6_SE_n7.csv ../../data/oxford_buildings/flatfiles/oxford_buildings_flat50_6classes.csv ../../data/oxford_buildings/mini_batch/metrics/
ObjectID(98840b760a174a58ffffffff010000c801000000)
oxford_buildings_size200_c6_UMAP_n5_d0.8.csv
/Users/morarica/Developer/DumbleDR/public/data/oxford_buildings/human_sample_mini/oxford_buildings_size200_c6_UMAP_n5_d0.8.csv ../../data/oxford_buildings/flatfiles/oxford_buildings_flat50_6classes.csv ../../data/oxford_buildings/mini_batch/metrics/
ObjectID(e0332cb2bfd80556

ObjectID(b567c3deb09019a4ffffffff010000c801000000)
paris_buildings_size50_c6_UMAP_n7_d0.8.csv
/Users/morarica/Developer/DumbleDR/public/data/paris_buildings/human_sample_mini/paris_buildings_size50_c6_UMAP_n7_d0.8.csv ../../data/paris_buildings/flatfiles/paris_buildings_flat50_6classes.csv ../../data/paris_buildings/mini_batch/metrics/
ObjectID(52fd7069fa50fb85ffffffff010000c801000000)
paris_buildings_size50_c6_UMAP_n5_d0.1.csv
/Users/morarica/Developer/DumbleDR/public/data/paris_buildings/human_sample_mini/paris_buildings_size50_c6_UMAP_n5_d0.1.csv ../../data/paris_buildings/flatfiles/paris_buildings_flat50_6classes.csv ../../data/paris_buildings/mini_batch/metrics/
ObjectID(73f81750cc1bb8f5ffffffff010000c801000000)
paris_buildings_size50_c6_TSNE_p5.csv
/Users/morarica/Developer/DumbleDR/public/data/paris_buildings/human_sample_mini/paris_buildings_size50_c6_TSNE_p5.csv ../../data/paris_buildings/flatfiles/paris_buildings_flat50_6classes.csv ../../data/paris_buildings/mini_batch/metri

(pid=97802) Starting... highlow_MNIST_size28_c6_ISM_n5
(pid=97800) Starting... highlow_MNIST_size28_c6_GRP
(pid=97801) Starting... highlow_MNIST_size28_c6_UMAP_n2_d0.1
(pid=97804) Starting... highlow_MNIST_size28_c6_UMAP_n15_d0.1
(pid=97807) Starting... highlow_MNIST_size28_c6_UMAP_n7_d0.8
(pid=97803) Starting... highlow_MNIST_size28_c6_ISM_n7
(pid=97806) Starting... highlow_MNIST_size28_c6_TSNE_p30
(pid=97805) Starting... highlow_MNIST_size28_c6_UMAP_n15_d0.5
(pid=97800) (7176, 786) (300, 4)
(pid=97802) (7176, 786) (300, 4)
(pid=97800) (300, 786) (300, 4)
(pid=97801) (7176, 786) (300, 4)
(pid=97802) (300, 786) (300, 4)
(pid=97801) (300, 786) (300, 4)
(pid=97804) (7176, 786) (300, 4)
(pid=97803) (7176, 786) (300, 4)
(pid=97804) (300, 786) (300, 4)
(pid=97807) (7176, 786) (300, 4)
(pid=97803) (300, 786) (300, 4)
(pid=97806) (7176, 786) (300, 4)
(pid=97807) (300, 786) (300, 4)
(pid=97806) (300, 786) (300, 4)
(pid=97805) (7176, 786) (300, 4)
(pid=97805) (300, 786) (300, 4)
(pid=97802) AUC

(pid=97815) AUClogRNX complete for: highlow_MNIST_size28_c6_ISM_n2
(pid=97815) CCA complete for: highlow_MNIST_size28_c6_ISM_n2
(pid=97815) CC complete for: highlow_MNIST_size28_c6_ISM_n2
(pid=97801) AUClogRNX complete for: highlow_MNIST_size28_c6_UMAP_n2_d0.5
(pid=97801) CCA complete for: highlow_MNIST_size28_c6_UMAP_n2_d0.5
(pid=97801) CC complete for: highlow_MNIST_size28_c6_UMAP_n2_d0.5
(pid=97805) AUClogRNX complete for: highlow_MNIST_size28_c6_MDS
(pid=97805) CCA complete for: highlow_MNIST_size28_c6_MDS
(pid=97805) CC complete for: highlow_MNIST_size28_c6_MDS
(pid=97803) AUClogRNX complete for: highlow_MNIST_size28_c6_PCA
(pid=97803) CCA complete for: highlow_MNIST_size28_c6_PCA
(pid=97803) CC complete for: highlow_MNIST_size28_c6_PCA
(pid=97807) AUClogRNX complete for: highlow_MNIST_size28_c6_ISM_n15
(pid=97800) AUClogRNX complete for: highlow_MNIST_size28_c6_UMAP_n5_d0.5
(pid=97807) CCA complete for: highlow_MNIST_size28_c6_ISM_n15
(pid=97800) CCA complete for: highlow_MNIST_s

In [4]:

apply_highlow('/Users/morarica/Developer/DumbleDR/public/data/MNIST/human_sample_mini/MNIST_size28_c6_ISM_n5.csv',
              '../../data/MNIST/flatfiles/MNIST_flat28_6classes.csv',
              '../../data/MNIST/mini_batch/metrics/')




Starting... highlow_MNIST_size28_c6_ISM_n5
(7176, 786) (300, 4)
(300, 786) (300, 4)
AUClogRNX complete for: highlow_MNIST_size28_c6_ISM_n5
CCA complete for: highlow_MNIST_size28_c6_ISM_n5
CC complete for: highlow_MNIST_size28_c6_ISM_n5
LCMC complete for: highlow_MNIST_size28_c6_ISM_n5
NeRV complete for: highlow_MNIST_size28_c6_ISM_n5
NLM complete for: highlow_MNIST_size28_c6_ISM_n5
Trust complete for: highlow_MNIST_size28_c6_ISM_n5
Cont complete for: highlow_MNIST_size28_c6_ISM_n5
Stress complete for: highlow_MNIST_size28_c6_ISM_n5


{'highlow_MNIST_size28_c6_ISM_n5': {'auclog': 0.0006209799221881764,
  'cca': 339.14486886996934,
  'cc': 0.021598154991798027,
  'lcmc': 1425486.7532892874,
  'nerv': 1.419235092948616e-16,
  'nlm': 2804782036844127.5,
  'stress': 0.9989395252514537,
  'trustworthiness': 13.519879534504602,
  'continuity': 13.39649008656708}}

In [5]:
# create mini batches
for f in all_names: 
    output_folder = '../../data/{}/mini_batch/'.format(f)
    
    try:
        os.makedirs(output_folder + "metrics/")
        os.makedirs(output_folder + "flatfiles/")
        os.makedirs(output_folder + "dr/")
    except:
        continue
        
    
    flat_files = '../../data/{}/flatfiles/'.format(f)
    for flat in os.listdir(flat_files):
        if not flat.endswith('.csv'):
            continue
        print(flat) 
        idx = sample_and_save(flat_files + flat, output_folder + "flatfiles/", size_limit = 800)

        if 'classes' not in flat:
            c = -1
            size = flat.split('flat')[-1].split('.csv')[0]
        else: 
            tmp = flat.split('.csv')[0]
            tmp = tmp.split('_')
            print(tmp)

            size = tmp[-2].split('flat')[-1]
            print(size)
            c = tmp[-1].split('classes')[0]
            print(c)

        dr_files = '../../data/{}/dr_data/size{}/'.format(f, size)
        for dr in os.listdir(dr_files):
            if not dr.endswith('.csv'):
                continue
                
            if ('c{}'.format(c) in dr):
                print(dr)
                sample_and_save(dr_files + dr, output_folder + "dr/", size_limit = 800, idx=idx)
            
            elif ('call' in dr) and (c==-1):
                print(dr)
                sample_and_save(dr_files + dr, output_folder + "dr/", size_limit = 800, idx=idx)
                
    

In [29]:
high = pd.read_csv('../../data/stanfordfaces/mini_batch/flatfiles/stanfordfaces_flat50.csv')
dr = pd.read_csv('../../data/stanfordfaces/mini_batch/dr/stanfordfaces_size50_call_UMAP_n15_d0.5.csv')
dr.shape
dr

,UMAP_1,UMAP_2,labels,filename
0,7.503400,6.276552,left down,left down_74_74
1,5.383214,3.682751,mid left up,mid left up_317_317
2,0.621888,7.902173,mid right down,mid right down_374_374
3,4.269809,7.556913,mid left down,mid left down_276_276
4,-5.283817,7.042769,right down,right down_568_568
...,...,...,...,...
693,10.299421,11.949068,left down,left down_14_14
694,9.005250,4.143293,left up,left up_166_166
695,4.775728,5.389145,mid left up,mid left up_350_350
696,7.601295,9.128738,mid left down,mid left down_231_231


In [58]:
%time
path = '../../data/stanfordfaces/mini_batch/dr/stanfordfaces_size50_call_UMAP_n15_d0.5.csv'
res = apply_supervised(path)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs
AWB complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5
CAL complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5
DSC complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5
HM complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5
NH complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5
SC complete for: supervised_stanfordfaces_size50_call_UMAP_n15_d0.5


In [83]:
%time
path = '../../data/stanfordfaces/mini_batch/dr/stanfordfaces_size50_call_UMAP_n15_d0.5.csv'
res = apply_scagnostics(path)
res

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
/Users/morarica/Developer/DRflow/DRflow/examples


{'scagnostics_stanfordfaces_size50_call_UMAP_n15_d0.5': {'outlying': 0.0,
  'skewed': 0.6364482601161021,
  'clumpy': 0.024160320958999695,
  'sparse': 0.027796075460412125,
  'striated': 0.06747404844290658,
  'convex': 0.49258801328782537,
  'skinny': 0.7894510877773224,
  'stringy': 0.34934344879754786,
  'monotonic': 0.1744664112159387}}

In [77]:
%time
from datetime import datetime as time
start = time.now()
dr_path = '../../data/stanfordfaces/mini_batch/dr/stanfordfaces_size50_call_UMAP_n15_d0.5.csv'
flat_path = '../../data/stanfordfaces/mini_batch/flatfiles/stanfordfaces_flat50.csv'
res_dr = apply_unsupervised(dr_path, flat_path)
end = time.now()
print(end-start)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
AUClogRNX complete for: stanfordfaces_size50_call_UMAP_n15_d0.5.csv
CCA complete for: stanfordfaces_size50_call_UMAP_n15_d0.5.csv


KeyboardInterrupt: 

In [20]:
path = '../../data/stanfordfaces/mini_batch/'
for folder in os.listdir(path):
    if not os.path.isdir(path + folder):
        continue
    ff = path + folder
    for f in os.listdir(ff):
        if not f.endswith('.csv'):
            continue
        print(f)
        file_path = ff + '/' + f
        df = pd.read_csv(file_path)
        df.labels = [i[0] for i in df.labels.str.split('_')]
        df.to_csv(file_path, index=False)  

stanfordfaces_size50_call_UMAP_n5_d0.5.csv
stanfordfaces_size50_call_UMAP_n15_d0.1.csv
stanfordfaces_size50_call_UMAP_n15_d0.5.csv
stanfordfaces_size50_call_UMAP_n5_d0.1.csv
stanfordfaces_size50_call_UMAP_n7_d0.8.csv
stanfordfaces_size50_call_TSNE_p30.csv
stanfordfaces_size50_call_UMAP_n2_d0.1.csv
stanfordfaces_size50_call_UMAP_n2_d0.5.csv
stanfordfaces_size50_call_TSNE_p100.csv
stanfordfaces_size50_call_TSNE_p5.csv
stanfordfaces_size50_call_SE_n7.csv
stanfordfaces_size50_call_UMAP_n7_d0.1.csv
stanfordfaces_size50_call_ISM_n2.csv
stanfordfaces_size50_call_TSNE_p10.csv
stanfordfaces_size50_call_UMAP_n2_d0.8.csv
stanfordfaces_size50_call_ISM_n7.csv
stanfordfaces_size50_call_UMAP_n7_d0.5.csv
stanfordfaces_size50_call_ISM_n15.csv
stanfordfaces_size50_call_ISM_n5.csv
stanfordfaces_size50_call_PCA.csv
stanfordfaces_size50_call_MDS.csv
stanfordfaces_size50_call_UMAP_n15_d0.8.csv
stanfordfaces_size50_call_GRP.csv
stanfordfaces_size50_call_UMAP_n5_d0.8.csv
stanfordfaces_flat50_10classes.csv
sta